---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [142]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [210]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    

    import pandas as pd

    import re

    with open('university_towns.txt', 'r') as in_file:
        with open('u_towns.txt', 'w') as out_file:
            for line in in_file:
                line = re.sub('\[[A-Za-z1-9]*\]', '', line)
                line = re.sub('\(.*\)', '', line)
                line = re.sub('.*:', '', line)
                line = re.sub('\(.*', '', line)
                line = re.sub('\[[0-9]*\]', '', line)
                line =re.sub('University Park, Los Angeles', 'Los Angeles', line)
                line = re.sub('Pomona  and formerly Pomona College', 'Pomona', line)
                line = re.sub('University District, San Bernardino', 'San Bernardino', line)
                line = re.sub('Pueblo, Colorado', 'Pueblo', line)
                line = re.sub('Cedar Rapids, Iowa', 'Cedar Rapids', line)
                line = re.sub('Mount Vernon,', 'Mount Vernon', line)
                change = 'Lewiston, Maine' 
                line = re.sub(change, 'Lewiston', line)
                change = 'Faribault, South Central College'
                line = re.sub(change, 'Faribault', line)
                change = 'Mankato , Bethany Lutheran College'
                line = re.sub(change, 'Mankato', line)
                change = 'North Mankato, South Central College'
                line = re.sub(change, 'North Mankato', line)
                change = 'New London, New Hampshire'
                line = re.sub(change, 'New London', line)
                change = 'Morningside Heights, Manhattan'
                line = re.sub(change, 'Manhattan', line)
                change = 'Wilmington, North Carolina'
                line = re.sub(change, 'Wilmington', line)
                change = 'Cecil B. Moore, Philadelphia, also known as \"Templetown\"'
                line = re.sub(change, 'Philadelphia', line)
                change = 'University City, Philadelphia'
                line = re.sub(change, 'Philadelphia', line)
                change = 'Oakland, Pittsburgh'
                line = re.sub(change, 'Pittsburgh', line)
                change = 'University District, Seattle'
                line = re.sub(change, 'Seattle', line)
                out_file.write(line)


    df = pd.read_csv('u_towns.txt', sep=' ', header=None, names=['Cities'])

    def update_state(name):
        states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
        if name in states.values():
            return name
        else:
            return "city"


    df['state'] = df['Cities'].apply(lambda x: update_state(x))

    state_reg = []
    
    def add_state(value):
        if value in states_in_list:
            state_reg.append(value)
            return value
        elif value == 'city':
            return state_reg[-1]

    df['state2'] = df['state'].apply(lambda x: add_state(x))

    df_2 = df.copy()

    df_3 = df_2.drop(df[df['state']==df['state2']].index)

    df_4 = df_3.drop('state', axis=1)

    df_5 = df_4[['state2', 'Cities']]

    df_6 = df_5.rename(columns={'state2': 'State', 'Cities': 'RegionName'})
    
    return df_6

In [211]:
get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [61]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    


    import pandas as pd

    df = pd.read_excel('gdplev.xls', header=None, skiprows=8)

    df_q = df[[4,6]]

    df_q = df_q.rename(columns={4: "Year, Quarter", 6: 'GDP'})


    df_2000 = df_q[212:]

    df_2000['GDP_change']=df_2000['GDP'].diff()

    return df_2000['Year, Quarter'].loc[246]

In [62]:
get_recession_start()

/Users/aschharwood/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'2008q3'

In [65]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       


    import pandas as pd

    df = pd.read_excel('gdplev.xls', header=None, skiprows=8)

    df_q = df[[4,6]]

    df_q = df_q.rename(columns={4: "Year, Quarter", 6: 'GDP'})

    df_2000 = df_q[212:]

    df_2000['GDP_change']=df_2000['GDP'].diff()

    return df_2000['Year, Quarter'].loc[251]

In [66]:
get_recession_end()

/Users/aschharwood/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'2009q4'

In [90]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    


    import pandas as pd

    df = pd.read_excel('gdplev.xls', header=None, skiprows=8)

    df_q = df[[4,6]]

    df_q = df_q.rename(columns={4: "Year, Quarter", 6: 'GDP'})

    df_2000 = df_q[212:]

    df_2000['GDP_change']=df_2000['GDP'].diff()

    recession_bottom = min(df_2000.loc[246:251]['GDP'])


    return df_2000.loc[df_2000['GDP'] == recession_bottom]['Year, Quarter'].tolist()[0]

In [91]:
get_recession_bottom()

/Users/aschharwood/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'2009q2'

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [92]:
import pandas as pd

In [100]:
df = pd.read_csv('City_Zhvi_AllHomes.csv')

In [101]:
df.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [143]:
regions = list(df.columns[0:6])
years = list(df.columns[51:])
column_names = regions + years

In [144]:
df_2 = df[column_names]

In [146]:
df_2.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,2000-01,2000-02,2000-03,2000-04,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,204400.0,207000.0,209800.0,212300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,136800.0,138300.0,140100.0,141900.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,52700.0,53100.0,53200.0,53400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,111000.0,111700.0,112800.0,113700.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [148]:
df_test = pd.DataFrame([[1,1,1,5],[1,1,2,5],[2,1,3,5]],columns=['a','a','a','a'])

idx = pd.date_range('20130101',periods=4,freq='Q-NOV')

df_test.columns = idx

In [150]:
df_test.columns

DatetimeIndex(['2013-02-28', '2013-05-31', '2013-08-31', '2013-11-30'], dtype='datetime64[ns]', freq='Q-NOV')

In [ ]:
idx = pd.datetime()

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"